In [1]:
import os

In [2]:
pwd

'd:\\MLOPs-Tutorials\\DS-Project-end2end\\research'

In [3]:
os.chdir('../')
%pwd

'd:\\MLOPs-Tutorials\\DS-Project-end2end'

## **DATA INGESTION ( updating config.yaml file!!)**

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.dataScience.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.dataScience.utils.common import read_yaml, create_directory

In [6]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH, 
                 schema_filepath=SCHEMA_FILE_PATH):
        
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        
        create_directory([self.config['artifacts_root']])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config['data_ingestion']
        create_directory([config['root_dir']])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config['root_dir'], 
            source_URL=config['source_URL'],  # fixed typo
            local_data_file=config['local_data_file'], 
            unzip_dir=config['unzip_dir']
        )
        return data_ingestion_config


In [7]:
from urllib import request
from src.dataScience import logger
import zipfile

#component-data-ingestion
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config
        
    def donwload_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers=request.urlretrieve(
                url=self.config.source_URL, 
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} download! with following info: \n{headers}')
        else:
            logger.info(f'file already exists')
    
    def extract_zip(self):
            unzip_path=self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            

In [9]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.donwload_file()
    data_ingestion.extract_zip()
except Exception as e:
    raise e

[2025-08-05 21:37:13,557: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2025-08-05 21:37:13,559: INFO: common: yaml file: params.yaml loaded succesfully]
[2025-08-05 21:37:13,559: INFO: common: yaml file: schema.yaml loaded succesfully]
[2025-08-05 21:37:13,562: INFO: common: created directory at: artifacts]
[2025-08-05 21:37:13,565: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-05 21:37:15,034: INFO: 653420479: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2520:C0D1C:14EDDF:2C7735:68922C33
Accept-Ranges: bytes
Date: Tue, 05 Aug 